In [1]:
import time
import numpy as np
import pandas as pd
import pickle
import datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import json
import dateparser

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('headless')    
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service, options=options)

In [41]:
class KommersantParser:
    '''Парсит коммерсант по конкректному региону
    driver - selenium.webdriver, который будете использовать
    day - дата, с какой будем начинать выкачивать новости
    interval: 1 - за день, 2 - новости за неделю, 3 - за месяц с сегодняшней даты
    По умолчанию качает понедельно (interval=week), по региону Урал (region=66), начиная с сегодня'''

    intervals = {'day': '1', 'week': '2', 'month': '3'}

    def __init__(self, driver: webdriver, interval='week', region=66, day=datetime.datetime.today()):
        self.driver = driver
        self.interval = str(interval)
        self.region = str(region)
        self.day = day
    
    
    def __get_page_content(self, api: str) -> BeautifulSoup:
        '''Открывает ссылку, парсит html'''
        
        self.driver.get(api) # открываем ссылку
        time.sleep(2) # даем прогрузиться
        page_source = self.driver.page_source # берем страницу
        soup = BeautifulSoup(page_source, "html5lib") # парсим

        return soup


    def __get_article(self, link: str) -> str:
        '''Открывает страничку статьи, собирает текст с нее'''
        
        while True:
            try:
                soup = self.__get_page_content(link)
                # здесь лежит статья
                if soup.find('div', class_='event'):
                    return ''
                ps = soup.find_all('p', class_='doc__text') # собираем все аттрибуты p, абзацы
                break
            except (AttributeError, TimeoutException):
                time.sleep(2)
                continue
                
        # две последние строчки в ps содержат слова подпишитесь и имя автора, они нам не нужны
        text = ''.join([p.text for p in ps[0:len(ps)-2]]).strip() 

        return text
        
    

    def __parse_first_page(self, soup: BeautifulSoup) -> list:
        '''Возвращает список собранных новостей с распрасенной страницы'''

        news = []

        rubrics = soup.find_all('article', class_='uho rubric_lenta__item js-article') # аттрибут новости
        for rubric in rubrics:
            cur_news = {}
            cur_news['description'] = rubric['data-article-description']
            cur_news['news_id'] = rubric['data-article-docsid']
            cur_news['title'] = rubric['data-article-title']
            cur_news['url'] = rubric['data-article-url']
            # дата
            news_date = dateparser.parse(rubric.find('p', class_='uho__tag rubric_lenta__item_tag hide_desktop').text)
            cur_news['published_date'] = news_date
            cur_news['text'] = self.__get_article(cur_news['url'])
            news.append(cur_news)

        return news
        
    
    def open_first_page(self) -> list:
        '''Открывает страницу архива новостей Коммерсанта с заданными вам параметрами из self, 
        возращает список собранных новостей'''

        today_for_starter_link = self.day.strftime('%Y-%m-%d')
        url = f'https://www.kommersant.ru/archive/list/{self.region}/{self.interval}/{today_for_starter_link}'
        soup = self.__get_page_content(url)

        return self.__parse_first_page(soup)

    
    def __parse_lazy_downloads(self, items: json) -> list:
        '''парсит страничку lazydownloads, достает атрибуты, загружает текст каждой новости'''
        parsed_news = []
        
        for item in items:
            cur_news = {}
            cur_news['description'] = item['SubTitle']
            cur_news['news_id'] = item['DocsID']
            cur_news['title'] = item['Title']
            cur_news['url'] = 'https://www.kommersant.ru/doc/' + str(item['DocsID'])
            cur_news['published_date'] = pd.to_datetime(item['DateDoc'], unit='s')
            cur_news['text'] = self.__get_article(cur_news['url'])
            parsed_news.append(cur_news)

        return parsed_news
       
    
    def __lazy_downloads(self, news: list, last_date: str) -> json:
        '''Делает запрос на lazydownloads с последней новости, 
        на которой остановился список выкачанных новостей, либо на последней со стратовой страницы.
        Возвращает обновленный список и отдельно состояние has_next_page и no_items, чтобы итерироваться'''

        last_id = news[-1]['news_id']
        url = f'https://www.kommersant.ru/archive/list/lazyloaddocs?regionid={self.region}&date={last_date}&intervaltype={self.intervals[self.interval]}&idafter={last_id}'
        print(url)
        soup = self.__get_page_content(url)
        data = json.loads(soup.text)
        news.extend(self.__parse_lazy_downloads(data['Items']))
        has_next_page = data['HasNextPage']
        no_items = False if data['Items'] else True

        return has_next_page, no_items

    
    def download_until_date(self, news: list, stop_date=datetime.datetime(2010,1,1)):
        '''Подгружает новости через lazydownloads до тех пор, пока не дойдет до стоп даты'''
        last_date = news[-1]['published_date'].date()
        stop_date = stop_date.date()
        
        while last_date >= stop_date:
            has_next_page, no_items = True, False
            while has_next_page:
                has_next_page, no_items = self.__lazy_downloads(news, last_date.strftime('%d.%m.%Y'))
                if no_items:
                    last_date -= datetime.timedelta(days=1)
                    has_next_page = True
                    continue
            with open(f'kommersant_{self.region}', 'wb') as f:
                pickle.dump(news, f)
            print('Updated')
            last_date = news[-1]['published_date'].date()

        print(f'Update a file up to {stop_date.strftime("%d.%m.%Y")}')

In [42]:
parser = KommersantParser(driver=driver, interval='week', region=66)
parser.download_until_date(news, stop_date=datetime.datetime(2010, 1, 1))

https://www.kommersant.ru/archive/list/lazyloaddocs?regionid=66&date=08.01.2019&intervaltype=2&idafter=3849553
True
https://www.kommersant.ru/archive/list/lazyloaddocs?regionid=66&date=07.01.2019&intervaltype=2&idafter=3849553
True
https://www.kommersant.ru/archive/list/lazyloaddocs?regionid=66&date=06.01.2019&intervaltype=2&idafter=3849553
True
https://www.kommersant.ru/archive/list/lazyloaddocs?regionid=66&date=05.01.2019&intervaltype=2&idafter=3849553
True
https://www.kommersant.ru/archive/list/lazyloaddocs?regionid=66&date=04.01.2019&intervaltype=2&idafter=3849553
False
Updated
https://www.kommersant.ru/archive/list/lazyloaddocs?regionid=66&date=29.12.2018&intervaltype=2&idafter=3848583
False
https://www.kommersant.ru/archive/list/lazyloaddocs?regionid=66&date=29.12.2018&intervaltype=2&idafter=3844309
False
https://www.kommersant.ru/archive/list/lazyloaddocs?regionid=66&date=29.12.2018&intervaltype=2&idafter=3843547
False
https://www.kommersant.ru/archive/list/lazyloaddocs?regionid

KeyboardInterrupt: 

In [45]:
news[-1]

{'description': '',
 'news_id': 3849553,
 'title': 'Пострадавших в ДТП югорчан могут направить на лечение в Екатеринбург',
 'url': 'https://www.kommersant.ru/doc/3849553',
 'published_date': Timestamp('2019-01-08 09:12:56'),
 'text': ''}

In [ ]:
del parser
del news

In [ ]:
parser_1 = KommersantParser(driver=driver, interval='week', region=2)
news_1 =  parser_1.open_first_page()
parser_1.download_until_date(news_1, stop_date=datetime.datetime(2010, 1, 1))

In [ ]:
del parser_1
del news_1

In [ ]:
parser_2 = KommersantParser(driver=driver, interval='week', region=74)
news_2 =  parser_2.open_first_page()
parser_2.download_until_date(news_2, stop_date=datetime.datetime(2010, 1, 1))

In [ ]:
del parser_2
del news_2

In [ ]:
parser_3 = KommersantParser(driver=driver, interval='week', region=59)
news_3 =  parser_3.open_first_page()
parser_3.download_until_date(news_3, stop_date=datetime.datetime(2010, 1, 1))

In [ ]:
del parser_3
del news_3

In [51]:
df = pd.DataFrame.from_records(news)

In [52]:
df[df['text'] == '']

,description,news_id,title,url,published_date,text
48,Фестиваль современной драматургии «Коляда-Play...,5422494,Пена театральных дней,https://www.kommersant.ru/doc/5422494,2022-06-20 18:15:32,
101,Выставка в Екатеринбургском музее изобразитель...,5421661,«Родное искусство»,https://www.kommersant.ru/doc/5421661,2022-06-18 11:17:25,
167,,5412745,ПСБ заключил соглашение о намерениях с Российс...,https://www.kommersant.ru/doc/5412745,2022-06-16 06:29:39,
179,Уральский оркестр сыграл Шуберта на высоте 55 ...,5411822,Музыка поднялась над городом,https://www.kommersant.ru/doc/5411822,2022-06-15 16:53:06,
319,Международный IT-форум в Ханты-Мансийске,5394910,Югра в цифровой повестке,https://www.kommersant.ru/doc/5394910,2022-06-09 05:06:56,
...,...,...,...,...,...,...
29475,,3849766,Екатеринбургская мэрия потратит на отлов бродя...,https://www.kommersant.ru/doc/3849766,2019-01-09 06:31:59,
29476,,3849756,«Автомобилист» вышел в плей-офф КХЛ,https://www.kommersant.ru/doc/3849756,2019-01-09 06:01:20,
29477,,3849755,В начале рабочей недели в Свердловской области...,https://www.kommersant.ru/doc/3849755,2019-01-09 05:59:09,
29478,,3849555,"Свердловское МЧС наградит школьника, спасшего ...",https://www.kommersant.ru/doc/3849555,2019-01-08 09:26:35,


In [53]:
df

,description,news_id,title,url,published_date,text
0,,5423649,«Уральские авиалинии» увеличат число рейсов из...,https://www.kommersant.ru/doc/5423649,2022-06-22 13:54:00,Авиакомпания «Уральские авиалинии» с 10 июля у...
1,,5423632,В Свердловской области за сутки подтвердили 91...,https://www.kommersant.ru/doc/5423632,2022-06-22 13:30:00,За прошедшие сутки в Свердловской области коро...
2,,5423605,СвЖД к маю 2023 года намерена создать выставоч...,https://www.kommersant.ru/doc/5423605,2022-06-22 12:50:00,Свердловская железная дорога (СвЖД) начала под...
3,,5423592,ФК «Урал» начнет новый сезон игрой против ЦСКА,https://www.kommersant.ru/doc/5423592,2022-06-22 12:36:00,В первом матче нового сезона Российской премье...
4,,5423571,Брянского подростка будут судить за лжеминиров...,https://www.kommersant.ru/doc/5423571,2022-06-22 12:22:00,Следствие завершило расследование уголовного д...
...,...,...,...,...,...,...
29475,,3849766,Екатеринбургская мэрия потратит на отлов бродя...,https://www.kommersant.ru/doc/3849766,2019-01-09 06:31:59,
29476,,3849756,«Автомобилист» вышел в плей-офф КХЛ,https://www.kommersant.ru/doc/3849756,2019-01-09 06:01:20,
29477,,3849755,В начале рабочей недели в Свердловской области...,https://www.kommersant.ru/doc/3849755,2019-01-09 05:59:09,
29478,,3849555,"Свердловское МЧС наградит школьника, спасшего ...",https://www.kommersant.ru/doc/3849555,2019-01-08 09:26:35,


In [4]:
with open('kommersant_66', 'rb') as f:
    news = pickle.load(f)

In [15]:
news[-1]['published_date'].date()

datetime.date(2019, 1, 8)

In [8]:
x = 5
a = True if x > 7 else False

In [9]:
a

False

In [16]:
datetime.date(1998,12,30) - datetime.timedelta(days=1)

datetime.date(1998, 12, 29)